## Aligning Study Results

In [27]:
from collections import Counter
import pandas as pd
import numpy as np
import krippendorff
pd.set_option('display.max_colwidth', None)

In [4]:
study_1_df = pd.read_csv('../../../docs/Claim_Annotation_Study_part_12021-07-12_07_03_01.csv')
study_2_df = pd.read_csv('../../../docs/Claim_Annotation_Study_part_22021-07-12_07_51_20.csv')
study_3_df = pd.read_csv('../../../docs/Claim_Annotation_Study_part_32021-07-12_16_55_48.csv')
study_4_df = pd.read_csv('../../../docs/Claim_Annotation_Study_part_42021-07-12_16_53_27.csv')

In [5]:
study_1_df['Submission Date'] = study_1_df['Submission Date'].apply(lambda x: x[:-3])
study_2_df['Submission Date'] = study_2_df['Submission Date'].apply(lambda x: x[:-3])
study_3_df['Submission Date'] = study_3_df['Submission Date'].apply(lambda x: x[:-3])
study_4_df['Submission Date'] = study_4_df['Submission Date'].apply(lambda x: x[:-3])

In [6]:
study_1_df.drop_duplicates(['Submission Date','Submission IP'],inplace=True)
study_2_df.drop_duplicates(['Submission Date','Submission IP'],inplace=True)
study_3_df.drop_duplicates(['Submission Date','Submission IP'],inplace=True)
study_4_df.drop_duplicates(['Submission Date','Submission IP'],inplace=True)

study_1_df.fillna('nan', inplace=True)
study_2_df.fillna('nan', inplace=True)
study_3_df.fillna('nan', inplace=True)
study_4_df.fillna('nan', inplace=True)

In [7]:
print(study_1_df.shape)
print(study_2_df.shape)
print(study_3_df.shape)
print(study_4_df.shape)

(6, 229)
(6, 227)
(6, 229)
(6, 229)


In [8]:
columns_lists = [study_1_df.columns,study_2_df.columns,study_3_df.columns,study_4_df.columns]

In [9]:
all_columns = list(set(list(study_1_df.columns)+list(study_2_df.columns)+list(study_3_df.columns)+list(study_4_df.columns)))

In [10]:
columns = list(set.intersection(*[set(x) for x in columns_lists]))

In [11]:
list(set(all_columns) - set(columns))

['Last Update Date', 'Submission ID']

In [12]:
study_1_df.loc[study_1_df['Submission Date']=='2021/07/11 18:17','Submission IP'] = study_1_df[study_1_df['Submission Date']=='2021/07/11 18:17']['Submission IP']+'_2'
study_2_df.loc[study_2_df['Submission Date']=='2021/07/11 20:03','Submission IP'] = study_2_df[study_2_df['Submission Date']=='2021/07/11 20:03']['Submission IP']+'_2'
study_3_df.loc[study_3_df['Submission Date']=='2021/07/11 20:27','Submission IP'] = study_3_df[study_3_df['Submission Date']=='2021/07/11 20:27']['Submission IP']+'_2'
study_4_df.loc[study_4_df['Submission Date']=='2021/07/11 19:27','Submission IP'] = study_4_df[study_4_df['Submission Date']=='2021/07/11 19:27']['Submission IP']+'_2'

study_3_df.loc[study_3_df['Submission IP']=='95.222.29.112','Submission IP'] = '94.114.198.82'
study_4_df.loc[study_4_df['Submission IP']=='95.222.29.112','Submission IP'] = '94.114.198.82'

In [13]:
ip_lists = [study_1_df['Submission IP'].tolist(),study_2_df['Submission IP'].tolist(),study_3_df['Submission IP'].tolist(),study_4_df['Submission IP'].tolist()]

In [14]:
ips = list(set.intersection(*[set(x) for x in ip_lists]))

In [15]:
ips = set(ips) - set(['93.231.74.101'])

In [16]:
answer_columns = list(set(all_columns) - set(['Submission ID', 'Last Update Date', 'Submission IP', 'Submission Date']))

In [17]:
study_1_df = study_1_df[study_1_df['Submission IP'].isin(ips)]
study_1_df.drop(['Submission ID', 'Last Update Date', 'Submission Date'], axis=1, inplace=True)
study_2_df = study_2_df[study_2_df['Submission IP'].isin(ips)]
study_2_df.drop(['Submission Date'], axis=1, inplace=True)
study_3_df = study_3_df[study_3_df['Submission IP'].isin(ips)]
study_3_df.drop(['Submission ID', 'Last Update Date', 'Submission Date'], axis=1, inplace=True)
study_4_df = study_4_df[study_4_df['Submission IP'].isin(ips)]
study_4_df.drop(['Submission ID', 'Last Update Date', 'Submission Date'], axis=1, inplace=True)

In [18]:
for column in study_2_df.columns:
    if column in answer_columns:
        study_2_df.rename(columns={column: str(int(column.split('.')[0])+25)+'.'+'.'.join(column.split('.')[1:])}, inplace=True)
for column in study_3_df.columns:
    if column in answer_columns:
        study_3_df.rename(columns={column: str(int(column.split('.')[0])+50)+'.'+'.'.join(column.split('.')[1:])}, inplace=True)
for column in study_4_df.columns:
    if column in answer_columns:
        study_4_df.rename(columns={column: str(int(column.split('.')[0])+75)+'.'+'.'.join(column.split('.')[1:])}, inplace=True)

In [19]:
merged_df = pd.merge(study_1_df, study_2_df, on="Submission IP")
merged_df = pd.merge(merged_df, study_3_df, on="Submission IP")
merged_df = pd.merge(merged_df, study_4_df, on="Submission IP")

In [20]:
merged_df.head()

,Submission IP,2.2.1 Are the premises sufficient to draw the conclusion?,2.2.2 How likely is it that the claim will be inferred from the context?,1.2.1 Are the premises sufficient to draw the conclusion?,19.3.3 How can the claim be composed from the context?,5.3.1 Are the premises sufficient to draw the conclusion?,6.2.1 Are the premises sufficient to draw the conclusion?,17.3.3 How can the claim be composed from the context?,5.1.1 Are the premises sufficient to draw the conclusion?,5.3.2 How likely is it that the claim will be inferred from the context?,...,76.3.2 How likely is it that the claim will be inferred from the context?,100.1.3 How can the claim be composed from the context?,87.3.2 How likely is it that the claim will be inferred from the context?,76.3.3 How can the claim be composed from the context?,76.1.1 Are the premises sufficient to draw the conclusion?,77.1.1 Are the premises sufficient to draw the conclusion?,77.1.2 How likely is it that the claim will be inferred from the context?,76.1.2 How likely is it that the claim will be inferred from the context?,77.1.3 How can the claim be composed from the context?,76.1.3 How can the claim be composed from the context?
0,95.222.24.213,1 (not sufficient),3 (possible),4 (rather sufficient),4 (inference),4 (rather sufficient),2 (rather insufficient),4 (inference),3 (undecided),4 (likely),...,4 (likely),4 (inference),3 (possible),4 (inference),2 (rather insufficient),1 (not sufficient),2 (unlikely),4 (likely),4 (inference),4 (inference)
1,37.201.154.50,4 (rather sufficient),4 (likely),4 (rather sufficient),4 (inference),4 (rather sufficient),4 (rather sufficient),5 (can't be composed),3 (undecided),4 (likely),...,3 (possible),4 (inference),3 (possible),4 (inference),3 (undecided),3 (undecided),3 (possible),3 (possible),4 (inference),4 (inference)
2,93.225.53.68_2,3 (undecided),4 (likely),5 (sufficient),3 (copying + fusion),3 (undecided),2 (rather insufficient),4 (inference),5 (sufficient),3 (possible),...,5 (very likely),4 (inference),3 (possible),2 (synonymous copying),4 (rather sufficient),3 (undecided),3 (possible),4 (likely),4 (inference),2 (synonymous copying)
3,93.225.53.68,3 (undecided),2 (unlikely),4 (rather sufficient),4 (inference),4 (rather sufficient),4 (rather sufficient),4 (inference),2 (rather insufficient),4 (likely),...,3 (possible),4 (inference),3 (possible),3 (copying + fusion),3 (undecided),3 (undecided),4 (likely),4 (likely),4 (inference),4 (inference)
4,94.114.198.82,3 (undecided),3 (possible),4 (rather sufficient),4 (inference),1 (not sufficient),4 (rather sufficient),4 (inference),3 (undecided),1 (very unlikely),...,4 (likely),4 (inference),4 (likely),4 (inference),4 (rather sufficient),4 (rather sufficient),3 (possible),3 (possible),4 (inference),4 (inference)


In [21]:
study_df = pd.read_csv('../../../docs/study_new.csv', sep=';', encoding='latin-1', nrows=100)

In [22]:
k=0
for i, row in study_df.iterrows():
    tmp_columns = list(merged_df.columns[[True if x[0]==str(k+1) else False for x in merged_df.columns.str.split('.').tolist()]])
    new_col_names = {}
    for column in tmp_columns:
        if column.split('.')[1] == '1':
            tmp_column = column.replace(".1.","."+str(int(row['id1'])+1)+".")
        elif column.split('.')[1] == '2':
            tmp_column = column.replace(".2.","."+str(int(row['id2'])+1)+".")
        elif column.split('.')[1] == '3':
            tmp_column = column.replace(".3.","."+str(int(row['id3'])+1)+".")
        else:
            print('error')
        new_col_names[column] = tmp_column
    merged_df.rename(columns=new_col_names, inplace=True)
    k+=1

In [23]:
study_df.head()

,Column1,index,local_sufficency,gt,supervised,unsupervised,premises,first,second,third,id1,id2,id3
0,0,essay145_1,0,"<p>The first and foremost reason lies in an inevitable fact that <span style=""background-color: #fdf2d0;"">the more advanced facilities we have, the more chances we can increase our knowledge</span>. For example, <span style=""background-color: #fdf2d0;"">library is a good place for students to study or research</span>. <span style=""background-color: #fdf2d0;"">They can concentrate on the subjects with a tranquil ambience; also, there are a lot of references for students to utilize</span>. <span style=""background-color: #fdf2d0;"">With this convenient and versatile facility, students can improve their learning condition</span>. Therefore, we can commence to agree that <span style=""background-color: #d8d2e7;"">certain facilities are beneficial to students</span>. </p> <p style=""text-align: right;""><span style=""color: #d8d2e7;"">???</span> = claim&nbsp; &nbsp; &nbsp;<span style=""color: #fdf2d0;"">???</span> = premise</p>","<p>The first and foremost reason lies in an inevitable fact that <span style=""background-color: #fdf2d0;"">the more advanced facilities we have, the more chances we can increase our knowledge</span>. For example, <span style=""background-color: #fdf2d0;"">library is a good place for students to study or research</span>. <span style=""background-color: #fdf2d0;"">They can concentrate on the subjects with a tranquil ambience; also, there are a lot of references for students to utilize</span>. <span style=""background-color: #fdf2d0;"">With this convenient and versatile facility, students can improve their learning condition</span>. Therefore, we can commence to agree that <span style=""background-color: #d8d2e7;""> the more advanced facilities we have, the better we can learn</span>. </p> <p style=""text-align: right;""><span style=""color: #d8d2e7;"">???</span> = claim&nbsp; &nbsp; &nbsp;<span style=""color: #fdf2d0;"">???</span> = premise</p>","<p>The first and foremost reason lies in an inevitable fact that <span style=""background-color: #fdf2d0;"">the more advanced facilities we have, the more chances we can increase our knowledge</span>. For example, <span style=""background-color: #fdf2d0;"">library is a good place for students to study or research</span>. <span style=""background-color: #fdf2d0;"">They can concentrate on the subjects with a tranquil ambience; also, there are a lot of references for students to utilize</span>. <span style=""background-color: #fdf2d0;"">With this convenient and versatile facility, students can improve their learning condition</span>. Therefore, we can commence to agree that <span style=""background-color: #d8d2e7;"">library is the best place to study or research </span>. </p> <p style=""text-align: right;""><span style=""color: #d8d2e7;"">???</span> = claim&nbsp; &nbsp; &nbsp;<span style=""color: #fdf2d0;"">???</span> = premise</p>","['the more advanced facilities we have, the more chances we can increase our knowledge', 'library is a good place for students to study or research', 'They can concentrate on the subjects with a tranquil ambience; also, there are a lot of references for students to utilize', 'With this convenient and versatile facility, students can improve their learning condition']","<p>The first and foremost reason lies in an inevitable fact that <span style=""background-color: #fdf2d0;"">the more advanced facilities we have, the more chances we can increase our knowledge</span>. For example, <span style=""background-color: #fdf2d0;"">library is a good place for students to study or research</span>. <span style=""background-color: #fdf2d0;"">They can concentrate on the subjects with a tranquil ambience; also, there are a lot of references for students to utilize</span>. <span style=""background-color: #fdf2d0;"">With this convenient and versatile facility, students can improve their learning condition</span>. Therefore, we can commen

In [24]:
merged_df.head()

,Submission IP,2.1.1 Are the premises sufficient to draw the conclusion?,2.1.2 How likely is it that the claim will be inferred from the context?,1.1.1 Are the premises sufficient to draw the conclusion?,19.1.3 How can the claim be composed from the context?,5.1.1 Are the premises sufficient to draw the conclusion?,6.2.1 Are the premises sufficient to draw the conclusion?,17.1.3 How can the claim be composed from the context?,5.3.1 Are the premises sufficient to draw the conclusion?,5.1.2 How likely is it that the claim will be inferred from the context?,...,76.3.2 How likely is it that the claim will be inferred from the context?,100.1.3 How can the claim be composed from the context?,87.1.2 How likely is it that the claim will be inferred from the context?,76.3.3 How can the claim be composed from the context?,76.1.1 Are the premises sufficient to draw the conclusion?,77.1.1 Are the premises sufficient to draw the conclusion?,77.1.2 How likely is it that the claim will be inferred from the context?,76.1.2 How likely is it that the claim will be inferred from the context?,77.1.3 How can the claim be composed from the context?,76.1.3 How can the claim be composed from the context?
0,95.222.24.213,1 (not sufficient),3 (possible),4 (rather sufficient),4 (inference),4 (rather sufficient),2 (rather insufficient),4 (inference),3 (undecided),4 (likely),...,4 (likely),4 (inference),3 (possible),4 (inference),2 (rather insufficient),1 (not sufficient),2 (unlikely),4 (likely),4 (inference),4 (inference)
1,37.201.154.50,4 (rather sufficient),4 (likely),4 (rather sufficient),4 (inference),4 (rather sufficient),4 (rather sufficient),5 (can't be composed),3 (undecided),4 (likely),...,3 (possible),4 (inference),3 (possible),4 (inference),3 (undecided),3 (undecided),3 (possible),3 (possible),4 (inference),4 (inference)
2,93.225.53.68_2,3 (undecided),4 (likely),5 (sufficient),3 (copying + fusion),3 (undecided),2 (rather insufficient),4 (inference),5 (sufficient),3 (possible),...,5 (very likely),4 (inference),3 (possible),2 (synonymous copying),4 (rather sufficient),3 (undecided),3 (possible),4 (likely),4 (inference),2 (synonymous copying)
3,93.225.53.68,3 (undecided),2 (unlikely),4 (rather sufficient),4 (inference),4 (rather sufficient),4 (rather sufficient),4 (inference),2 (rather insufficient),4 (likely),...,3 (possible),4 (inference),3 (possible),3 (copying + fusion),3 (undecided),3 (undecided),4 (likely),4 (likely),4 (inference),4 (inference)
4,94.114.198.82,3 (undecided),3 (possible),4 (rather sufficient),4 (inference),1 (not sufficient),4 (rather sufficient),4 (inference),3 (undecided),1 (very unlikely),...,4 (likely),4 (inference),4 (likely),4 (inference),4 (rather sufficient),4 (rather sufficient),3 (possible),3 (possible),4 (inference),4 (inference)


---

## IAA agreement

### Question 1

In [30]:
mapped_labels = {
    '1 (not sufficient)': 0.0,
    '2 (rather insufficient)': 1.0,
    '3 (undecided)': 2.0,
    '4 (rather sufficient)': 3.0,
    '5 (sufficient)': 4.0,
    '1 (very unlikely)': 0.0,
    '2 (unlikely)': 1.0,
    '3 (possible)': 2.0,
    '4 (likely)': 3.0,
    '5\xa0\xa0(very likely)': 4.0,
    '1 (verbatim copying)': 0.0,
    '2 (synonymous copying)': 1.0,
    '3 (copying + fusion)': 2.0,
    '4 (inference)': 3.0,
    """5 (can't be composed)""": 4.0,
    'nan': 2.0
}

In [34]:
for i in range(3):
    merged_df_q1 = merged_df.loc[:, merged_df.columns.str.contains('{}.1 '.format(i+1), regex=False)]
    reliability_data = merged_df_q1.values[:,1:]
    reliability_data = np.vectorize(mapped_labels.get)(reliability_data)
    print("Krippendorff's alpha for model{}: ".format(i+1), krippendorff.alpha(reliability_data=reliability_data,
                                                                          level_of_measurement='ordinal'))

Krippendorff's alpha for model1:  0.23050232444204177
Krippendorff's alpha for model2:  0.3477454607110909
Krippendorff's alpha for model3:  0.4298167128914223


### Question 2

In [41]:
mapped_labels = {
    '1 (not sufficient)': 0.0,
    '2 (rather insufficient)': 1.0,
    '3 (undecided)': 2.0,
    '4 (rather sufficient)': 3.0,
    '5 (sufficient)': 4.0,
    '1 (very unlikely)': 0.0,
    '2 (unlikely)': 1.0,
    '3 (possible)': 2.0,
    '4 (likely)': 3.0,
    '5\xa0\xa0(very likely)': 4.0,
    '1 (verbatim copying)': 0.0,
    '2 (synonymous copying)': 1.0,
    '3 (copying + fusion)': 2.0,
    '4 (inference)': 3.0,
    """5 (can't be composed)""": 4.0,
    'nan': 0.0
}

In [42]:
for i in range(3):
    merged_df_q1 = merged_df.loc[:, merged_df.columns.str.contains('{}.2 '.format(i+1), regex=False)]
    reliability_data = merged_df_q1.values[:,1:]
    reliability_data = np.vectorize(mapped_labels.get)(reliability_data)
    print("Krippendorff's alpha for model{}: ".format(i+1), krippendorff.alpha(reliability_data=reliability_data,
                                                                          level_of_measurement='ordinal'))

Krippendorff's alpha for model1:  0.18556529799611676
Krippendorff's alpha for model2:  0.2771922760715524
Krippendorff's alpha for model3:  0.31821540074263976


### Question 3

In [43]:
mapped_labels = {
    '1 (not sufficient)': 0.0,
    '2 (rather insufficient)': 1.0,
    '3 (undecided)': 2.0,
    '4 (rather sufficient)': 3.0,
    '5 (sufficient)': 4.0,
    '1 (very unlikely)': 0.0,
    '2 (unlikely)': 1.0,
    '3 (possible)': 2.0,
    '4 (likely)': 3.0,
    '5\xa0\xa0(very likely)': 4.0,
    '1 (verbatim copying)': 0.0,
    '2 (synonymous copying)': 1.0,
    '3 (copying + fusion)': 2.0,
    '4 (inference)': 3.0,
    """5 (can't be composed)""": 4.0,
    'nan': 4.0
}

In [44]:
for i in range(3):
    merged_df_q1 = merged_df.loc[:, merged_df.columns.str.contains('{}.3 '.format(i+1), regex=False)]
    reliability_data = merged_df_q1.values[:,1:]
    reliability_data = np.vectorize(mapped_labels.get)(reliability_data)
    print("Krippendorff's alpha for model{}: ".format(i+1), krippendorff.alpha(reliability_data=reliability_data,
                                                                          level_of_measurement='ordinal'))

Krippendorff's alpha for model1:  0.19077952248709684
Krippendorff's alpha for model2:  0.3159853569426163
Krippendorff's alpha for model3:  0.500648667928762


---

## Majority agreement

In [45]:
zero_cols = []
for column in merged_df.columns:
    if '0' in merged_df[column].values:
        zero_cols.append(column)
merged_df.drop(zero_cols, axis=1, inplace=True)

In [46]:
merged_df = merged_df.replace({
    '1 (not sufficient)': 0,
    '2 (rather insufficient)': 1,
    '3 (undecided)': 2,
    '4 (rather sufficient)': 3,
    '5 (sufficient)': 4,
})
merged_df = merged_df.replace({
    '1 (very unlikely)': 0,
    '2 (unlikely)': 1,
    '3 (possible)': 2,
    '4 (likely)': 3,
    '5\xa0\xa0(very likely)': 4,
})
merged_df = merged_df.replace({
    '1 (verbatim copying)': 0,
    '2 (synonymous copying)': 1,
    '3 (copying + fusion)': 2,
    '4 (inference)': 3,
    """5 (can't be composed)""": 4,
})

In [47]:
study_df['idx'] = np.arange(len(study_df))

In [48]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 2 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    if max(counts)>(len(x)/2):
    #if counts_sum>10:
        return 1
    else:
        return 0

In [49]:
merged_df_ls = merged_df.loc[:, merged_df.columns.str.contains('.1 ', regex=False)]

In [50]:
study_df['majority_agreement_ls'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_ls[str(x+1)+'.1.1 Are the premises sufficient to draw the conclusion?'].tolist()) if str(x+1)+'.1.1 Are the premises sufficient to draw the conclusion?' in merged_df_ls.columns else None)
print(len(study_df[study_df['majority_agreement_ls']==1])/(len(study_df[study_df['majority_agreement_ls']==1])+len(study_df[study_df['majority_agreement_ls']==0])))
study_df['majority_agreement_ls'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_ls[str(x+1)+'.2.1 Are the premises sufficient to draw the conclusion?'].tolist()) if str(x+1)+'.2.1 Are the premises sufficient to draw the conclusion?' in merged_df_ls.columns else None)
print(len(study_df[study_df['majority_agreement_ls']==1])/(len(study_df[study_df['majority_agreement_ls']==1])+len(study_df[study_df['majority_agreement_ls']==0])))
study_df['majority_agreement_ls'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_ls[str(x+1)+'.3.1 Are the premises sufficient to draw the conclusion?'].tolist()) if str(x+1)+'.3.1 Are the premises sufficient to draw the conclusion?' in merged_df_ls.columns else None)
print(len(study_df[study_df['majority_agreement_ls']==1])/(len(study_df[study_df['majority_agreement_ls']==1])+len(study_df[study_df['majority_agreement_ls']==0])))

0.63
0.6
0.63


In [51]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 0 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    if max(counts)>len(x)/2:
    #if counts_sum>10:
        return 1
    else:
        return 0

In [52]:
merged_df_like = merged_df.loc[:, merged_df.columns.str.contains('.2 ', regex=False)]

In [53]:
study_df['majority_agreement_like'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_like[str(x+1)+'.1.2 How likely is it that the claim will be inferred from the context?'].tolist()) if str(x+1)+'.1.2 How likely is it that the claim will be inferred from the context?' in merged_df_like.columns else None)
print(len(study_df[study_df['majority_agreement_like']==1])/(len(study_df[study_df['majority_agreement_like']==1])+len(study_df[study_df['majority_agreement_like']==0])))
study_df['majority_agreement_like'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_like[str(x+1)+'.2.2 How likely is it that the claim will be inferred from the context?'].tolist()) if str(x+1)+'.2.2 How likely is it that the claim will be inferred from the context?' in merged_df_like.columns else None)
print(len(study_df[study_df['majority_agreement_like']==1])/(len(study_df[study_df['majority_agreement_like']==1])+len(study_df[study_df['majority_agreement_like']==0])))
study_df['majority_agreement_like'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_like[str(x+1)+'.3.2 How likely is it that the claim will be inferred from the context?'].tolist()) if str(x+1)+'.3.2 How likely is it that the claim will be inferred from the context?' in merged_df_like.columns else None)
print(len(study_df[study_df['majority_agreement_like']==1])/(len(study_df[study_df['majority_agreement_like']==1])+len(study_df[study_df['majority_agreement_like']==0])))

0.57
0.64
0.64


In [54]:
merged_df_novelty = merged_df.loc[:, merged_df.columns.str.contains('.3 ', regex=False)]

In [55]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 4 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    if max(counts)>len(x)/2:
    #if counts_sum>10:
        return 1
    else:
        return 0

In [56]:
study_df['majority_agreement_novelty'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_novelty[str(x+1)+'.1.3 How can the claim be composed from the context?'].tolist()) if str(x+1)+'.1.3 How can the claim be composed from the context?' in merged_df_novelty.columns else None)
print(len(study_df[study_df['majority_agreement_novelty']==1])/(len(study_df[study_df['majority_agreement_novelty']==1])+len(study_df[study_df['majority_agreement_novelty']==0])))
study_df['majority_agreement_novelty'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_novelty[str(x+1)+'.2.3 How can the claim be composed from the context?'].tolist()) if str(x+1)+'.2.3 How can the claim be composed from the context?' in merged_df_novelty.columns else None)
print(len(study_df[study_df['majority_agreement_novelty']==1])/(len(study_df[study_df['majority_agreement_novelty']==1])+len(study_df[study_df['majority_agreement_novelty']==0])))
study_df['majority_agreement_novelty'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_novelty[str(x+1)+'.3.3 How can the claim be composed from the context?'].tolist()) if str(x+1)+'.3.3 How can the claim be composed from the context?' in merged_df_novelty.columns else None)
print(len(study_df[study_df['majority_agreement_novelty']==1])/(len(study_df[study_df['majority_agreement_novelty']==1])+len(study_df[study_df['majority_agreement_novelty']==0])))

0.82
0.72
0.8


---

## Majority Scores

In [41]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 2 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    winner = np.argwhere(counts == np.amax(counts)).flatten().tolist()
    if len(winner)>1:
        return int(np.mean(x))
    else:
        return winner[0]

In [42]:
merged_df_ls = merged_df.loc[:, merged_df.columns.str.contains('.1 ', regex=False)]

In [43]:
study_df['majority_agreement_ls'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_ls[str(x+1)+'.1.1 Are the premises sufficient to draw the conclusion?'].tolist()) if str(x+1)+'.1.1 Are the premises sufficient to draw the conclusion?' in merged_df_ls.columns else None)
print(study_df['majority_agreement_ls'].value_counts())
print(len(study_df['majority_agreement_ls']))
study_df['majority_agreement_ls'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_ls[str(x+1)+'.2.1 Are the premises sufficient to draw the conclusion?'].tolist()) if str(x+1)+'.2.1 Are the premises sufficient to draw the conclusion?' in merged_df_ls.columns else None)
print(study_df['majority_agreement_ls'].value_counts())
print(len(study_df['majority_agreement_ls']))
study_df['majority_agreement_ls'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_ls[str(x+1)+'.3.1 Are the premises sufficient to draw the conclusion?'].tolist()) if str(x+1)+'.3.1 Are the premises sufficient to draw the conclusion?' in merged_df_ls.columns else None)
print(study_df['majority_agreement_ls'].value_counts())
print(len(study_df['majority_agreement_ls']))

2    39
3    30
1    24
0     7
Name: majority_agreement_ls, dtype: int64
100
3    34
2    30
1    21
0    14
4     1
Name: majority_agreement_ls, dtype: int64
100
2    34
3    32
1    19
0    14
4     1
Name: majority_agreement_ls, dtype: int64
100


In [44]:
merged_df_like = merged_df.loc[:, merged_df.columns.str.contains('.2 ', regex=False)]

In [45]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 0 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    winner = np.argwhere(counts == np.amax(counts)).flatten().tolist()
    if len(winner)>1:
        return int(np.mean(x))
    else:
        return winner[0]

In [46]:
study_df['majority_agreement_like'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_like[str(x+1)+'.1.2 How likely is it that the claim will be inferred from the context?'].tolist()) if str(x+1)+'.1.2 How likely is it that the claim will be inferred from the context?' in merged_df_like.columns else None)
print(study_df['majority_agreement_like'].value_counts())
study_df['majority_agreement_like'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_like[str(x+1)+'.2.2 How likely is it that the claim will be inferred from the context?'].tolist()) if str(x+1)+'.2.2 How likely is it that the claim will be inferred from the context?' in merged_df_like.columns else None)
print(study_df['majority_agreement_like'].value_counts())
study_df['majority_agreement_like'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_like[str(x+1)+'.3.2 How likely is it that the claim will be inferred from the context?'].tolist()) if str(x+1)+'.3.2 How likely is it that the claim will be inferred from the context?' in merged_df_like.columns else None)
print(study_df['majority_agreement_like'].value_counts())

2    54
3    24
1    18
0     4
Name: majority_agreement_like, dtype: int64
2    38
3    35
1    15
0    12
Name: majority_agreement_like, dtype: int64
2    46
3    23
0    16
1    15
Name: majority_agreement_like, dtype: int64


In [47]:
merged_df_novelty = merged_df.loc[:, merged_df.columns.str.contains('.3 ', regex=False)]

In [48]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 4 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    winner = np.argwhere(counts == np.amax(counts)).flatten().tolist()
    if len(winner)>1:
        return int(np.mean(x))
    else:
        return winner[0]

In [49]:
study_df['majority_agreement_novelty'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_novelty[str(x+1)+'.1.3 How can the claim be composed from the context?'].tolist()) if str(x+1)+'.1.3 How can the claim be composed from the context?' in merged_df_novelty.columns else None)
print(study_df['majority_agreement_novelty'].value_counts())
study_df['majority_agreement_novelty'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_novelty[str(x+1)+'.2.3 How can the claim be composed from the context?'].tolist()) if str(x+1)+'.2.3 How can the claim be composed from the context?' in merged_df_novelty.columns else None)
print(study_df['majority_agreement_novelty'].value_counts())
study_df['majority_agreement_novelty'] = study_df['idx'].apply(lambda x: get_majority_ls(x, merged_df_novelty[str(x+1)+'.3.3 How can the claim be composed from the context?'].tolist()) if str(x+1)+'.3.3 How can the claim be composed from the context?' in merged_df_novelty.columns else None)
print(study_df['majority_agreement_novelty'].value_counts())

3    73
2    18
4     5
1     4
Name: majority_agreement_novelty, dtype: int64
3    53
2    22
4     9
1     8
0     8
Name: majority_agreement_novelty, dtype: int64
3    47
2    15
0    14
4    13
1    11
Name: majority_agreement_novelty, dtype: int64


---

## Ranks

### Question 1

In [57]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 2 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    winner = np.argwhere(counts == np.amax(counts)).flatten().tolist()
    if len(winner)>1:
        return int(np.mean(x))
    else:
        return winner[0]

In [58]:
tmp_df1 = merged_df.loc[:, merged_df.columns.str.contains('.{}.1'.format(1), regex=False)]
tmp_df2 = merged_df.loc[:, merged_df.columns.str.contains('.{}.1'.format(2), regex=False)]
tmp_df3 = merged_df.loc[:, merged_df.columns.str.contains('.{}.1'.format(3), regex=False)]

In [59]:
tmp_df1_cols = [x.replace('.1.1','.').split('.')[0] for x in tmp_df1.columns]
tmp_df2_cols = [x.replace('.2.1','.').split('.')[0] for x in tmp_df2.columns]
tmp_df3_cols = [x.replace('.3.1','.').split('.')[0] for x in tmp_df3.columns]

In [60]:
col_lists = [tmp_df1_cols,tmp_df2_cols,tmp_df3_cols]

In [61]:
cols = list(set.intersection(*[set(x) for x in col_lists]))

In [62]:
len(cols)

100

In [63]:
import scipy.stats as ss

In [64]:
majority_scores = {}
for tmp_df in [tmp_df1,tmp_df2,tmp_df3]:
    for col in tmp_df.columns:
        if col != 'Submission IP':
            if col.split('.')[0] in cols:
                if col.split('.')[2][0] =='1':
                    if col.split('.')[0] not in majority_scores:
                        majority_scores[col.split('.')[0]] = []
                    majority_scores[col.split('.')[0]].append(get_majority_ls(1,merged_df[col]))

In [65]:
for key, value in majority_scores.items():
    majority_scores[key] = ss.rankdata(value, method='dense')

In [67]:
abs(np.mean(list(majority_scores.values()),axis=0)-3)

array([1.43, 1.37, 1.4 ])

### Question 2

In [68]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 0 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    winner = np.argwhere(counts == np.amax(counts)).flatten().tolist()
    if len(winner)>1:
        return int(np.mean(x))
    else:
        return winner[0]

In [69]:
tmp_df1 = merged_df.loc[:, merged_df.columns.str.contains('.{}.2'.format(1), regex=False)]
tmp_df2 = merged_df.loc[:, merged_df.columns.str.contains('.{}.2'.format(2), regex=False)]
tmp_df3 = merged_df.loc[:, merged_df.columns.str.contains('.{}.2'.format(3), regex=False)]

In [70]:
len(tmp_df3_cols)

100

In [71]:
tmp_df1_cols = [x.replace('.1.2','.').split('.')[0] for x in tmp_df1.columns]
tmp_df2_cols = [x.replace('.2.2','.').split('.')[0] for x in tmp_df2.columns]
tmp_df3_cols = [x.replace('.3.2','.').split('.')[0] for x in tmp_df3.columns]

In [72]:
col_lists = [tmp_df1_cols,tmp_df2_cols,tmp_df3_cols]

In [73]:
cols = list(set.intersection(*[set(x) for x in col_lists]))

In [74]:
len(cols)

100

In [75]:
majority_scores = {}
for tmp_df in [tmp_df1,tmp_df2,tmp_df3]:
    for col in tmp_df.columns:
        if col != 'Submission IP':
            if col.split('.')[0] in cols:
                if col.split('.')[2][0] =='2':
                    if col.split('.')[0] not in majority_scores:
                        majority_scores[col.split('.')[0]] = []
                    majority_scores[col.split('.')[0]].append(get_majority_ls(1,merged_df[col]))

In [76]:
for key, value in majority_scores.items():
    majority_scores[key] = ss.rankdata(value, method='dense')

In [77]:
abs(np.mean(list(majority_scores.values()),axis=0)-3)

array([1.42, 1.39, 1.54])

### Question 3

In [78]:
def get_majority_ls(i, x):
    counts = np.zeros(len(x))
    values = np.arange(len(x))
    x = [x_ if x_ != 'nan' else 4 for x_ in list(x)]
    counts_sum = 0
    for x_ in x:
        counts[x_] += 1
        counts_sum += values[x_]
    winner = np.argwhere(counts == np.amax(counts)).flatten().tolist()
    if len(winner)>1:
        return int(np.mean(x))
    else:
        return winner[0]

In [79]:
tmp_df1 = merged_df.loc[:, merged_df.columns.str.contains('.{}.3'.format(1), regex=False)]
tmp_df2 = merged_df.loc[:, merged_df.columns.str.contains('.{}.3'.format(2), regex=False)]
tmp_df3 = merged_df.loc[:, merged_df.columns.str.contains('.{}.3'.format(3), regex=False)]

In [80]:
len(tmp_df3_cols)

100

In [81]:
tmp_df1_cols = [x.replace('.1.3','.').split('.')[0] for x in tmp_df1.columns]
tmp_df2_cols = [x.replace('.2.3','.').split('.')[0] for x in tmp_df2.columns]
tmp_df3_cols = [x.replace('.3.3','.').split('.')[0] for x in tmp_df3.columns]

In [82]:
col_lists = [tmp_df1_cols,tmp_df2_cols,tmp_df3_cols]

In [83]:
cols = list(set.intersection(*[set(x) for x in col_lists]))

In [84]:
len(cols)

100

In [85]:
majority_scores = {}
for tmp_df in [tmp_df1,tmp_df2,tmp_df3]:
    for col in tmp_df.columns:
        if col != 'Submission IP':
            if col.split('.')[0] in cols:
                if col.split('.')[2][0] =='3':
                    if col.split('.')[0] not in majority_scores:
                        majority_scores[col.split('.')[0]] = []
                    majority_scores[col.split('.')[0]].append(get_majority_ls(1,merged_df[col]))

In [86]:
for key, value in majority_scores.items():
    majority_scores[key] = ss.rankdata(value, method='dense')

In [87]:
abs(np.mean(list(majority_scores.values()),axis=0)-3)

array([1.39, 1.54, 1.54])